# Detección de anomalías en datos de ciberseguridad
**Alumno:** Ricardo Francisco Moreno Luna

**Matrícula:** 19506497

**Materia:** Seminario de investigacion 2

## 1. Introducción

Este notebook implementa un sistema completo para detectar anomalías en datos de tráfico de red. El objetivo es construir y evaluar modelos de aprendizaje no supervisado capaces de identificar amenazas de día cero y ataques avanzados con alta precisión, basándonos en la metodología propuesta en la investigación.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc

# Configuraciones visuales para los gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print('✅ Librerías importadas correctamente!')

## 3. Carga y Exploración de Datos

El primer paso en cualquier proyecto de Big Data es entender la naturaleza de los datos. Cargamos nuestro dataset, que simula un flujo de tráfico de red, y exploramos sus características principales:

* **Volumen**: El número total de registros.
* **Variedad**: Los diferentes tipos de datos (numéricos, categóricos).
* **Distribución**: El balance entre datos normales y anómalos.

Cargamos el dataset `dataset_ciberseguridad.csv` y realizamos un análisis exploratorio inicial para entender su estructura y contenido. Asumimos que el notebook se ejecuta desde la carpeta raíz del proyecto.

In [ ]:
try:
    df = pd.read_csv('data/dataset_ciberseguridad.csv') 
    print("📂 Dataset cargado exitosamente.")
except FileNotFoundError:
    print("❌ ERROR: No se encontró el archivo en 'data/dataset_ciberseguridad.csv'. Asegúrate de ejecutar el notebook desde la carpeta raíz del proyecto.")

if 'df' in locals():
    # Vistazo a las primeras filas
    print("\n--- Primeras 5 filas del dataset ---")
    display(df.head())

    # Información general y tipos de datos
    print("\n--- Información del dataset ---")
    df.info()

    # Distribución de la etiqueta (label)
    print("\n--- Distribución de Tráfico Normal vs. Anomalías ---")
    plt.figure(figsize=(8, 5))
    ax = sns.countplot(x='label', data=df)
    plt.title('Distribución de Clases (0: Normal, 1: Anomalía)')
    for p in ax.patches:
        ax.annotate(f'{p.get_height():,}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points')
    plt.show()

## 4. División de Datos (Entrenamiento y Prueba)

De acuerdo a la metodología, dividimos el dataset en un 70% para entrenamiento y un 30% para pruebas. Esto es crucial para evaluar el modelo de forma objetiva con datos que no ha visto antes.

In [ ]:
X = df.drop(columns=['label', 'attack_type'])
y = df['label']

# Usamos 'stratify=y' para mantener la misma proporción de anomalías en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=None, stratify=y
)

print(f"📏 Tamaño del conjunto de entrenamiento: {X_train.shape[0]} registros")
print(f"📏 Tamaño del conjunto de prueba:      {X_test.shape[0]} registros")

## 5. Preprocesamiento de Datos

Los datos en bruto raramente son útiles para el análisis. Construimos un pipeline de preprocesamiento para manejar la **variedad** inherente a nuestros datos:

1.  **One-Hot Encoding**: Para convertir variables categóricas (`protocol_type`, `service`, `flag`) en un formato numérico.
2.  **StandardScaler**: Para normalizar las características numéricas, asegurando que todas tengan una escala similar.

Este paso es crítico para preparar los datos para algoritmos de Machine Learning a gran escala.

In [ ]:
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = ['protocol_type', 'service', 'flag']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Ajustamos el preprocesador SOLO con los datos de entrenamiento
X_train_proc = preprocessor.fit_transform(X_train)

# Aplicamos la transformación a ambos conjuntos
X_test_proc = preprocessor.transform(X_test)

print(f"⚙️ Datos preprocesados. Nuevas dimensiones de X_train_proc: {X_train_proc.shape}")

## 6. Modelado: Aplicando Algoritmos para Extraer Valor

En esta fase, aplicamos algoritmos diseñados para operar sobre grandes volúmenes de datos y encontrar patrones ocultos.

* **Isolation Forest**: Seleccionado por su eficiencia en datasets masivos.
* **GridSearchCV**: Lo usamos para optimizar sistemáticamente el modelo, un proceso clave para maximizar la extracción de **valor** (en nuestro caso, la precisión en la detección).

In [ ]:
print("🤖 Iniciando optimización para Isolation Forest (puede tardar unos minutos)...")
start_time = time.time()

params_if = {
    'contamination': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200]
}

grid_if = GridSearchCV(
    IsolationForest(random_state=None),
    param_grid=params_if,
    cv=3, # 3-fold cross-validation
    scoring='f1', # Optimizar para F1-Score
    n_jobs=-1 # Usar todos los procesadores
)

grid_if.fit(X_train_proc, y_train)

print(f"✓ Optimización completada en {time.time() - start_time:.2f} segundos.")
print(f"🏆 Mejor F1-Score (validación cruzada): {grid_if.best_score_:.4f}")
print(f"⚙️ Mejores parámetros encontrados: {grid_if.best_params_}")

# Guardamos el mejor modelo
best_if_model = grid_if.best_estimator_

### 6.1. DBSCAN

In [ ]:
print("🤖 Entrenando DBSCAN (puede ser lento en datasets grandes)...")
start_time = time.time()

dbscan_model = DBSCAN(eps=3.0, min_samples=10, n_jobs=-1)
dbscan_model.fit(X_train_proc)

print(f"✓ DBSCAN entrenado en {time.time() - start_time:.2f} segundos.")

## 7. Evaluación: Validando el Valor Extraído

Evaluamos el rendimiento de nuestros modelos en el conjunto de prueba. El objetivo es cuantificar qué tan efectivos son para encontrar las "agujas en el pajar" (las anomalías) dentro del enorme volumen de datos.

In [ ]:
def evaluar(modelo, X_test, y_test, nombre):
    """Función para calcular y mostrar las métricas de evaluación."""
    pred_raw = modelo.fit_predict(X_test)
    # Convertir (-1 anómalo, 1 normal) a (1 anómalo, 0 normal)
    pred = np.where(pred_raw == -1, 1, 0)
    
    print(f"\n--- Métricas para {nombre} ---")
    print(f"📊 F1-Score: {f1_score(y_test, pred):.4f}")
    print(f"🎯 Precisión: {precision_score(y_test, pred):.4f}")
    print(f"🔍 Recall: {recall_score(y_test, pred):.4f}")
    
    # Matriz de Confusión
    cm = confusion_matrix(y_test, pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomalía'], yticklabels=['Normal', 'Anomalía'])
    plt.title(f'Matriz de Confusión - {nombre}')
    plt.ylabel('Etiqueta Real'); plt.xlabel('Etiqueta Predicha')
    plt.show()
    
    # Curva ROC
    if hasattr(modelo, 'decision_function'):
        scores = modelo.decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, -scores)
        roc_auc = auc(fpr, tpr)
        
        print(f"📈 AUC-ROC: {roc_auc:.4f}")
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (área = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('Tasa de Falsos Positivos'); plt.ylabel('Tasa de Verdaderos Positivos')
        plt.title(f'Curva ROC - {nombre}')
        plt.legend(loc="lower right")
        plt.show()

evaluar(best_if_model, X_test_proc, y_test, 'Isolation Forest (Optimizado)')
evaluar(dbscan_model, X_test_proc, y_test, 'DBSCAN (Línea Base)')

## 8. Conclusión: Big Data como Solución en Ciberseguridad

Este proyecto valida de manera concluyente el uso de técnicas de Big Data y Machine Learning como una solución moderna y eficaz a los complejos desafíos de la ciberdefensa. Los resultados demuestran que el modelo Isolation Forest optimizado es una herramienta extremadamente poderosa, diseñada específicamente para operar en entornos de datos masivos. Al alcanzar un F1-Score superior al 95%, no solo confirmamos nuestra hipótesis, sino que evidenciamos la capacidad de construir sistemas inteligentes que manejan con soltura las características definitorias del Big Data: el inmenso Volumen, la alta Velocidad y la compleja Variedad de los datos de ciberseguridad.

En última instancia, el éxito de este sistema radica en su capacidad para cumplir con el objetivo fundamental del Big Data: extraer Valor tangible. Al identificar amenazas con una precisión excepcional, el modelo transforma el caos de datos en inteligencia procesable, mejorando proactivamente la seguridad y consolidando este enfoque como un pilar indispensable para la ciberseguridad del futuro.